In [1]:
import pandas as pd
import numpy as np
import requests
from get_mail_mailru import main as mail_main
from parser_broker_report import main as parser_main

In [2]:
def fix_split(ticker_list, transactions, transactions_executed, share_split_dict):

    replacement_dict = {
        'VTBE': 'RSHE',
        'RU000A102HB1': 'SU26227RMFS7',
        'RU000A1038V6': 'SU26238RMFS4',
        'RU000A101QE0': 'SU26234RMFS3'
    }

    ticker_list_copy = ticker_list.copy()
    for ticker in ticker_list_copy:

        if ticker in replacement_dict:
            new_ticker = replacement_dict[ticker]
            ticker_list.remove(ticker)
            ticker_list.append(new_ticker)
            transactions_executed.loc[transactions['Код'] == ticker, 'Код'] = new_ticker

    for ticker in ticker_list:

        if ticker in share_split_dict:
            transactions_executed.loc[(transactions_executed['Код'] == ticker) &
                                      (transactions_executed['Дата заключения'] <
                                       share_split_dict[ticker][0]), 'Количество'] =\
                (transactions_executed.loc[(transactions_executed['Код'] == ticker) &
                                           (transactions_executed['Дата заключения'] <
                                           share_split_dict[ticker][0]), 'Количество'] *
                 share_split_dict[ticker][1])


In [3]:
def get_stock_data_dict(tickers: list, transactions_executed):

    round_numb_3_list = ['LKOH', 'MGNT']
    round_numb_4_list = ['IRAO', 'MOEX', 'YDEX', 'SBMM']
    round_numb_5_list = ['HYDR', 'AFKS']
    round_numb_6_list = ['GAZP', 'MTSS', 'NVTK', 'ROSN', 'SBER', 'CHMF', 'SNGS', 'SBGD', 'SBMX', 'AFKS', 'AFLT', 'RTKM']

    share_amount_dict = {}
    share_price_dict = {}
    share_commission_dict = {}
    for ticker in tickers:

        if ticker in round_numb_3_list:
            round_numb = 3
        elif ticker in round_numb_4_list:
            round_numb = 4
        elif ticker in round_numb_5_list:
            round_numb = 5
        elif ticker in round_numb_6_list:
            round_numb = 6
        else:
            round_numb = 2

        share_frame = transactions_executed[transactions_executed['Код'] == ticker]
        share_list = list(zip(share_frame['Вид'], share_frame['Количество'], share_frame['Сумма'],
                              share_frame['Комиссия Брокера'], share_frame['Комиссия Биржи']))

        share_amount = share_price_avg = share_total_cost = share_commission = 0
        for i in range(len(share_list)):

            share_type = share_list[i][0]
            amount_new = share_list[i][1]
            share_total_cost_new = share_list[i][2]
            share_commission_new = share_list[i][3] + share_list[i][4]

            if share_type == 'Покупка':
                share_amount += amount_new
                share_total_cost += share_total_cost_new
                share_commission += share_commission_new
                share_price_avg = round((share_total_cost + share_commission) / share_amount, round_numb)

            elif share_type == 'Продажа':
                share_amount -= amount_new
                share_total_cost = share_amount * share_price_avg

                if share_amount > 0:
                    share_commission += share_commission_new
                else:
                    share_commission = 0

            else:
                raise Exception('Неверный вид транзакции')

        share_amount_dict[ticker] = share_amount
        share_price_dict[ticker] = share_price_avg
        share_commission_dict[ticker] = round(share_commission, 2)

    return share_amount_dict, share_price_dict, share_commission_dict


In [4]:
def get_trading_dict():

    trading_dict = {}
    trading_mode_list = ['TQBR', 'TQTF', 'TQCB', 'TQOB', 'TQIR']
    for id_trading in trading_mode_list:

        stocks = 'shares'
        if id_trading in ['TQOB', 'TQCB', 'TQIR']:
            stocks = 'bonds'

        url = (f"https://iss.moex.com/iss/engines/stock/markets/{stocks}/boards/{id_trading}/" 
               f"securities.csv?iss.meta=off&iss.only=marketdata&marketdata.columns=SECID,LAST")
        try:
            csv_text = requests.get(url).text.split('\n')

        except requests.exceptions.ConnectionError as e:
            raise Exception('Проблема с запросом: ' + url)

        trading_dict[id_trading] = csv_text

    return trading_dict


In [5]:
def get_last_prices_dict(tickers: list):

    last_prices_dict = {}
    trading_dict = get_trading_dict()
    for ticker in tickers:

        if ticker == 'RU000A101FA1':
            ticker = 'SU25084RMFS3'
        if ticker == 'VTBE':
            ticker = 'RSHE'

        for value in trading_dict.values():
            for line in value:
                line = line.split(';')

                if ticker in line:
                    last_prices_dict[ticker] = line[1]

    return last_prices_dict


In [6]:
def get_coupon_dict(tickers: list):

    coupon_dict = {}
    url = "https://iss.moex.com/iss/engines/stock/markets/bonds/boards/TQOB/securities.csv?iss.meta=off&iss.only" \
          "=securities&securities.columns=SECID,ACCRUEDINT "
    csv_text = requests.get(url).text.split('\n')

    for ticker in tickers:
        for line in csv_text:
            line = line.split(';')
            if ticker in line:
                coupon_dict[ticker] = line[1]

    return coupon_dict


In [ ]:
# def get_coupon_dict(tickers: list):

#     coupon_dict = {}
#     for id_trading in ['TQOB', 'TQCB', 'TQIR']:
#         url = (f"https://iss.moex.com/iss/engines/stock/markets/bonds/boards/{id_trading}/securities.csv?iss.meta=off&iss.only/securities&securities.columns=SECID,ACCRUEDINT")

#         try:
#             csv_text = requests.get(url).text.split('\n')
#         except requests.exceptions.ConnectionError as e:
#             raise Exception('Проблема с запросом: ' + url)

#         for ticker in tickers:
#             for line in csv_text:
#                 line = line.split(';')
#                 if ticker in line:
#                     coupon_dict[ticker] = line[7]
#                     print(line)

#     return coupon_dict


In [7]:
# mail_main()
parser_main()

In [22]:
portfolio = pd.read_csv('portfolio.csv')
portfolio['Дата'] = portfolio['Дата'].astype('datetime64[ns]')
transactions = pd.read_csv('transactions.csv').drop_duplicates(['Дата заключения', 'Время заключения', 'Статус', 'Номер сделки'])
transactions['Дата заключения'] = pd.to_datetime(transactions['Дата заключения'] + ' ' + transactions['Время заключения'], dayfirst=True)
transactions['Дата расчетов'] = pd.to_datetime(transactions['Дата расчетов'], dayfirst=True)
transactions = transactions.drop('Время заключения', axis=1).dropna(axis=1).sort_values('Дата заключения').reset_index(drop=True)
transactions_executed = transactions[transactions['Статус'] == 'И']

In [70]:
fxus = transactions_executed[transactions_executed['Код'] == 'SU26238RMFS4']
fxus

,Дата заключения,Дата расчетов,Наименование,Код,Валюта,Вид,Количество,Цена,Сумма,НКД,Комиссия Брокера,Комиссия Биржи,Номер сделки,Статус
486,2024-07-18 11:15:24,2024-07-19,ОФЗ26238,SU26238RMFS4,RUB,Покупка,10,51.05,5105.0,85.6,3.06,0.51,10953509669,И


In [24]:
ticker_list = list(transactions_executed['Код'].unique())

share_split_dict = {
    'FXGD': ['2022-02-17', 10],
    'SBMX': ['2021-06-09', 100],
    'FXUS': ['2022-01-24', 100],
    'FXRL': ['2021-11-24', 100],
    'FXRU': ['2022-02-17', 10],
    'FXDE': ['2021-12-15', 100]
}


In [25]:
fix_split(ticker_list, transactions, transactions_executed, share_split_dict)
share_amount_dict, share_price_dict, share_commission_dict = get_stock_data_dict(ticker_list, transactions_executed)
last_prices_dict = get_last_prices_dict(ticker_list)
coupon_dict = get_coupon_dict(ticker_list)

portfolio_dict = {
    'Котировки': last_prices_dict,
    'НКД': coupon_dict,
    'Количество': share_amount_dict,
    'Средняя цена': share_price_dict,
    'Комиссия': share_commission_dict
}

main_df = pd.DataFrame.from_dict(portfolio_dict)
main_df.index.name = 'Название'
# main_df.loc[main_df['Котировки'] == '', 'Котировки'] = np.nan
# main_df.loc[main_df['Количество'] == 0, 'Количество'] = np.nan
main_df[(main_df['Количество'] == 0) | (main_df['Котировки'] == '')] = np.nan
# main_df = main_df.drop(main_df[main_df['Количество'] == 0].index)
# main_df = main_df.drop(main_df[main_df['Котировки'] == ''].index)
main_df.dropna(axis=0, subset=['Котировки', 'Количество'], inplace=True)
main_df[['Котировки', 'НКД']] = main_df[['Котировки', 'НКД']].astype('float64')
main_df['Текущая цена'] = main_df['Котировки'] * main_df['Количество']
main_df['P/L, руб.'] = main_df['Текущая цена'] - main_df['Средняя цена'] * main_df['Количество']
main_df['P/L, %'] = (main_df['Котировки'] * 100 / main_df['Средняя цена'] - 100).round(2)

# bond_price, bond_NKD, bond_amount, bond_average_price, bond_commission = \
#     main_df.loc['SU25084RMFS3', 'Котировки'], \
#     main_df.loc['SU25084RMFS3', 'НКД'], \
#     main_df.loc['SU25084RMFS3', 'Количество'], \
#     main_df.loc['SU25084RMFS3', 'Средняя цена'], \
#     main_df.loc['SU25084RMFS3', 'Комиссия']

# main_df.loc['SU25084RMFS3', 'Котировки'] = bond_price * 10
# main_df.loc['SU25084RMFS3', 'Текущая цена'] = (bond_price * 10 + bond_NKD) * bond_amount
# main_df.loc['SU25084RMFS3', 'P/L, руб.'] = (main_df.loc['SU25084RMFS3', 'Текущая цена'] -
#                                             bond_average_price * bond_amount)
# main_df.loc['SU25084RMFS3', 'P/L, %'] = (((bond_price * 10 + bond_NKD) * 100 / bond_average_price) - 100).round(2)

main_df.to_csv(path_or_buf='portfolio_main.csv', index_label=False)


['SU26238RMFS4', 'TQOB', 'ОФЗ 26238', '53.754', '15.13', '35.4', '2025-06-04', '8.17', '53.95', '1', '1000', 'Т+: Гособлигации - безадрес.', 'A', '2041-05-15', '4', '182', '500000000', '53.852', '2025-01-13', 'ОФЗ-ПД 26238 15/05/2041', '', 'FNDT', 'GOFZ', '', '0.001', 'SUR', '', '0000-00-00', 'RU000A1038V6', 'OFZ-PD 26238', '26238RMFS', 'SUR', '500000000', '1', '3', '7.100', '', '2025-01-15', '1000', '1000', '', '', '']
['SU26238RMFS4', '53.854', '', '53.881', '', '0.027', '382166', '174593', '53.95', '53.6', '54', '53.887', '0.038', '0.07', '1163', '626705.81', '15.09', '6101.81', '53.808', '0.133', '0.1', '0.054', '15.12', '-0.01', '0', '', '', '53.747', '-0.12', '4161', '634036', '341161431', '3321660', 'TQOB', 'T', '15:19:20', '2715', '', '', '-0.063', '15:19:14', '', '', '0.133', '', '', '53.859', '', '', '53.95', '20250114153421', '2025-01-14 15:34:21', '341161431', '', '', '', '', '', '1', '', '']
['SU26238RMFS4', 'TQOB', '53.791', '2041-05-15', '2025-01-14 15:19:53', 'MATDATE',

In [26]:
df = pd.read_csv('portfolio_main.csv')
df.index.name = 'Название'

In [20]:
df

,Котировки,НКД,Количество,Средняя цена,Комиссия,Текущая цена,"P/L, руб.","P/L, %"
Название,,,,,,,,
SBMX,18.0400,NaN,1400.0,17.312579,7.04,25256.00,1018.389400,4.20
SBER,280.3100,NaN,150.0,266.495667,33.45,42046.50,2072.149950,5.18
MTSS,214.4000,NaN,100.0,254.623000,21.30,21440.00,-4022.300000,-15.80
GAZP,133.7800,NaN,90.0,167.589667,13.57,12040.20,-3042.870030,-20.17
LKOH,7048.0000,NaN,4.0,6694.875000,17.50,28192.00,1412.500000,5.27
MGNT,4721.0000,NaN,5.0,5782.144000,23.72,23605.00,-5305.720000,-18.35
CHMF,1273.8000,NaN,14.0,1458.571429,17.00,17833.20,-2586.800006,-12.67
AFKS,14.8830,NaN,1200.0,17.475590,18.41,17859.60,-3111.108000,-14.84
IRAO,3.7650,NaN,4300.0,4.123200,14.74,16189.50,-1540.260000,-8.69


In [ ]:
fxus = transactions_executed[transactions_executed['Код'] == 'FXRL']